In [0]:
import gym
import numpy as np
from gym.envs.registration import register

## 카트폴 게임

In [0]:
#installing dependencies
!apt-get -qq -y install libnvtoolsext1 > /dev/null
!ln -snf /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so.8.0 /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so
!apt-get -qq -y install xvfb freeglut3-dev ffmpeg> /dev/null
!pip -q install gym
!pip -q install pyglet
!pip -q install pyopengl
!pip -q install pyvirtualdisplay

In [0]:
import gym
env = gym.make("CartPole-v0")

In [0]:
obs = env.reset()
print(f"initial observation", obs)

In [0]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1024, 768))
display.start()

In [0]:
np.set_printoptions(threshold=np.inf)

- 3, 3, 5, 5, 2로 나눴는데 각각 의미하는 바는 X, seta, dx/dt, d seta / dt 이고, 마지막 2는 Action을 의미

In [0]:
Q = np.zeros((3, 3, 5, 5, 2))

In [0]:
LOW_BOUNDS = [-1.0, -0.2, -0.2, -0.5]

In [0]:
HIGH_BOUNDS = [1.0, 0.2, 0.2, 0.5]

In [0]:
def state_to_bucket(state, debug):
    state = observation
    bucket_indice = []
    for i in range(4):
        if debug:
            print("="* 100)
            print("i = ", i)
            print(f"state[{i}] = {state[i]}")
            print(f"LOW_BOUNDS[{i}] = {LOW_BOUNDS[i]}")
            print(f"HIGH_BOUNDS[{i}] = {HIGH_BOUNDS[i]}")
            print(f"Q.shape[{i}] = {Q.shape[i]}")
        if state[i] <= LOW_BOUNDS[i]:
            bucket_index = 0
        elif state[i] >= HIGH_BOUNDS[i]:
            bucket_index = Q.shape[i] - 1
        else:
            scale = (HIGH_BOUNDS[i] - LOW_BOUNDS[i]) / Q.shape[i]
            bucket_index = int((state[i] - LOW_BOUNDS[i]) / scale)
            if debug:
                print(f"scale : {scale}")
                print(f"bucket_index = {bucket_index}")
        bucket_indice.append(bucket_index)
    if debug:
        print(f"bucket_indice = {bucket_indice}")
    return tuple(bucket_indice)

## 개선된 방식으로 학습하기
- 기존의 행동으로 학습된 내용이 있어도 다시 한번 수행해서 학습하도록 수정

In [0]:
import random as pr

dis = 1   # cartpole은 길면 길 수록 좋은 것이기 때문에 reward를 계속 증가시켜주는 것
Q = np.zeros((3, 3, 5, 5, 2))
for episode in range(1000):
    observation = env.reset()
    total_frame = 0
    for frame in range(200):
        state_0 = state_to_bucket(observation, False)

        e = 1.0/((episode // 100)+1)
        if np.random.rand(1) < e:
            action = pr.choice([0, 1])
        else:
            action = np.argmax(Q[state_0])

        observation, reward, done, info = env.step(action)
        # print(f"observation : {observation}, reward : {reward}, done:{done}, info : {info}")
        if done == True:
            break

        state = state_to_bucket(observation, False)

        best_q = np.amax(Q[state])

        Q[state_0 + (action, )] = reward + dis * best_q

        state_0 = state
        total_frame = frame
    print(total_frame)

In [0]:
Q

In [0]:
observation = env.reset()
animationFrame = []
for frame in range(200):
    animationFrame.append(
        env.render(mode = 'rgb_array')
    )
    state_0 = state_to_bucket(observation, False)

    m = np.amax(Q[state_0])

    if m == 0:
        action = pr.choice([0, 1])
    else:
        action = np.argmax(Q[state_0])

    
    observation, reward, done, info = env.step(action)

    if done == True:
        break


In [0]:
import matplotlib.pyplot as plt
import matplotlib.animation
import numpy as np
from IPython.display import HTML

## 카트폴 게임 이미지를 저장하고, 보여주기

In [0]:
def animate(index):
    patch.set_data(animationFrame[index])

In [0]:
len(animationFrame)

In [0]:
plt.figure(figsize=(8, 5))
patch = plt.imshow(animationFrame[0])
ani = matplotlib.animation.FuncAnimation(plt.gcf(), 
                                         animate, 
                                         frames=len(animationFrame),
                                         interval = 50)

HTML(ani.to_jshtml())